# Convert fore names to simplified (one word) fore names

In [1]:
import collections
import string
import pandas

import pubmedpy.names
from pubmedpy.names import simplify_fore_name, simplify_last_name

In [2]:
name_df_paths = [
    'data/pubmed/authors.tsv.xz',
    'data/pmc/authors.tsv.xz',
    'data/iscb/keynotes.tsv',
]
path_to_df = {}
for path in name_df_paths:
    name_df = pandas.read_csv(path, keep_default_na=False, na_values=[''], sep='\t')
    path_to_df[path] = name_df

## full name table

create a table of individuals with full names for prediction methods that require full names.

In [3]:
full_name_df = pandas.concat(
    [df[["fore_name", "last_name"]] for df in path_to_df.values()]
).drop_duplicates()
full_name_df.head(2)

,fore_name,last_name
0,A H,Samad
1,W W,Cai


In [4]:
def clean_full_name(name: str) -> str:
    """
    Lightly clean full names by replacing periods with spaces
    and standardizing whitespace.
    """
    name = name.replace(".", " ")
    name = " ".join(name.split())
    return name

In [5]:
full_name_df['fore_name_simple'] = full_name_df.fore_name.map(
    lambda x: simplify_fore_name(x, lower=True))
full_name_df['last_name_simple'] = full_name_df.last_name.map(
    lambda x: simplify_last_name(x, lower=True))
full_name_df = full_name_df.dropna(subset=['fore_name_simple', "last_name_simple"])
full_name_df["full_name"] = (
    full_name_df.fore_name.str.strip() + " " +
    full_name_df.last_name.str.strip()).map(clean_full_name)
full_name_df.sample(n=10, random_state=0)

,fore_name,last_name,fore_name_simple,last_name_simple,full_name
189136,Meng-Hua,Li,meng-hua,li,Meng-Hua Li
66524,Lee,Smith,lee,smith,Lee Smith
566731,Charles D,Searles,charles,searles,Charles D Searles
489139,Jessica L,Linville,jessica,linville,Jessica L Linville
321497,Edgar P,Spalding,edgar,spalding,Edgar P Spalding
345069,Jinhee,Kim,jinhee,kim,Jinhee Kim
653507,Le,Gu,le,gu,Le Gu
855604,Lucie,Hamel,lucie,hamel,Lucie Hamel
445990,Stephan,Bessler,stephan,bessler,Stephan Bessler
221313,Farhan A,Pasha,farhan,pasha,Farhan A Pasha


In [6]:
full_name_df.to_csv('data/names/full-names.tsv.xz', sep='\t', index=False)

## fore and last name tables

In [7]:
def simplify_names(dfs, name_column):
    names = pandas.concat(df[name_column] for df in dfs)
    simple_name_df = names.value_counts(dropna=True).reset_index()
    simple_name_df.columns = [name_column, 'n_authors']
    assert simple_name_df[name_column].isna().sum() == 0
    simplifier = getattr(pubmedpy.names, f'simplify_{name_column}')
    simple_name_df[f'{name_column}_simple'] = simple_name_df[name_column].map(lambda x: simplifier(x, lower=True))
    simple_name_df = simple_name_df.sort_values(name_column)
    return simple_name_df

In [8]:
simple_fore_name_df = simplify_names(path_to_df.values(), 'fore_name')
simple_fore_name_df

,fore_name,n_authors,fore_name_simple
139846,(Max) Zong-Ming,1,max
141671,", Yuxin",1,yuxin
106853,-,1,None
3,A,3763,None
123891,A Gordon,1,gordon
...,...,...,...
113424,Živojin,1,živojin
104510,Žygimantė,1,žygimantė
131563,željka,1,željka
77864,Νicolaos,1,νicolaos


In [9]:
simple_last_name_df = simplify_names(path_to_df.values(), 'last_name')
simple_last_name_df

,last_name,n_authors,last_name_simple
116533,'t Hart,1,t hart
5494,'t Hoen,26,t hoen
114987,(Holly) Yang,1,holly) yang
108257,-L Zhu,1,l zhu
9999,A,16,a
...,...,...,...
150669,žurauskienė,1,žurauskienė
168909,ʼt Hoen,1,ʼt hoen
146670,Ӧkmen,1,ӧkmen
76174,‘t Hoen,2,‘t hoen


In [10]:
simple_fore_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,fore_name,n_authors,fore_name_simple
11340,Juan-Carlos,14,juan-carlos
28587,Djamel,5,djamel
33178,Hui-Qing,4,hui-qing
31960,Diana C J,4,diana
36896,Furqan,4,furqan
37540,Shih-Hwa,4,shih-hwa
65117,Terence M,2,terence
52174,Hongxian,2,hongxian
52062,Sharadha,2,sharadha
129363,Oswaldo Keith,1,oswaldo


In [11]:
simple_last_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,last_name,n_authors,last_name_simple
8824,Regoes,17,regoes
15270,Lucena,11,lucena
18674,Ishino,9,ishino
21807,Jylhä,8,jylhä
24812,Gulcher,7,gulcher
31591,Stenbroen,5,stenbroen
42368,Dharmawardhana,4,dharmawardhana
45472,Er,4,er
56982,Baguelin,3,baguelin
53728,Moez,3,moez


In [12]:
# Most common last names
(
    simple_last_name_df
    .groupby('last_name_simple')
    .n_authors.sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(3)
)

,last_name_simple,n_authors
0,wang,19753
1,zhang,17130
2,li,16889


In [13]:
simple_fore_name_df.to_csv('data/names/fore-names.tsv.xz', sep='\t', index=False)
simple_last_name_df.to_csv('data/names/last-names.tsv.xz', sep='\t', index=False)